In [12]:
import numpy as np
from scipy.spatial.distance import pdist

In [52]:
from scipy.spatial import distance_matrix

In [2]:
def read_and_convert_data_points(filename):
    splits = []
    with open(filename, 'r') as f:
        data_points = f.readlines()
    
    for data_point in data_points:
        splits.append(data_point.split())
    
    return np.array(splits, dtype=np.float)

In [3]:
data_points = read_and_convert_data_points('Data_Exercise4_2019')

In [57]:
distance_matrix= distance_matrix(data_points,data_points)